In [1]:
using Pkg, Revise
Pkg.activate(joinpath(@__DIR__,"../.."))
using EasyABM

  Activating project at `~/Documents/EasyABM`
[ Info: Precompiling EasyABM [2ab8e4d3-e1e9-4248-86a3-8419d2d242e9]


WebIO._IJuliaInit()

In [2]:
balls = con_2d_agents(1000, pos = Vect(0.0,0.0), size = 0.05,mass=1.0,  keeps_record_of = Set([:pos]))
box = con_2d_agent(pos=Vect(5.0,5.0), shape = :square_line, size = 10.0)
model = create_2d_model([box, balls...], agents_type=Static, space_type = NPeriodic)

EasyABM SpaceModel2D{EasyABM.StaticType, Float64, EasyABM.NPeriodicType}: In a EasyABM.StaticType model number of agents is fixed.


In [3]:
function initialiser!(model)
    xdim, ydim = model.size
    #medium
    for i in 3:length(model.agents)
        agent=model.agents[i]
        agent.pos = Vect(rand()*xdim, rand()*ydim)
        v = Vect(rand()-0.5, rand()-0.5) 
        agent.vel = v/(veclength(v)+0.001)
        agent.last_pos = agent.pos
    end
    
    #brownian particle
    model.agents[2].size = 0.30 #
    model.agents[2].color = cl"blue"
    model.agents[2].pos = Vect(5.0,5.0)
    model.agents[2].vel = Vect(0.0,0.0)
    model.agents[2].mass = 10.0
    
end


init_model!(model, initialiser=initialiser!)

In [4]:
function components(vec1, dir)
    val = dotprod(vec1, dir)
    vec_parl = dir*val
    vec_perp = vec1-vec_parl
    return vec_parl, vec_perp   
end

function transfer_momentum(agent, nbrs)
    for nbr in nbrs
        dir = nbr.pos - agent.pos
        dist = veclength(dir)
        if (nbr.shape == :circle) && (dist < agent.size+nbr.size)
            unit_dir = dir/dist
            mag = (agent.size+nbr.size) - dist
            disp = unit_dir*mag*0.5
            agent.pos -= disp
            nbr.pos += disp
            ag_vel_parl, ag_vel_perp = components(agent.vel, unit_dir)
            nbr_vel_parl, nbr_vel_perp = components(nbr.vel, unit_dir)
            x = nbr.mass/agent.mass
            agent.vel = ag_vel_perp + (2*x*nbr_vel_parl+(1-x)*ag_vel_parl)/(1+x)
            nbr.vel = nbr_vel_perp + (2*ag_vel_parl+(x-1)*nbr_vel_parl)/(1+x)
        end
    end
end

function reflect_from_boundaries(agent,xdim,ydim,dw)
    x,y=agent.pos
    velx, vely = agent.vel
    if (x< dw) && (velx <0)
        velx =-velx
    end
    if (x> xdim - dw) && (velx > 0)
        velx =-velx
    end
    if (y< dw) && (vely <0)
        vely =-vely
    end
    if (y> ydim- dw) && (vely > 0)
        vely =-vely
    end
    agent.vel = Vect(velx, vely)
end

function step_rule!(model)
    xdim, ydim = model.size
    search_radius = model.agents[2].size # size of Brownian particle
    for i in 2:length(model.agents)
        agent=model.agents[i]
        reflect_from_boundaries(agent,xdim, ydim, search_radius)
        nbrs = neighbors(agent, model, 2*agent.size) # agent.size is the radius
        transfer_momentum(agent, nbrs)
        agent.pos += agent.vel*0.1
    end
end

step_rule! (generic function with 1 method)

In [5]:
run_model!(model, steps=200, step_rule = step_rule!)

In [6]:
animate_sim(model, tail=(200, agent-> agent.color==cl"blue"))

WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["time"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 201, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable(101), nothing)), Set{String}(), nothing, WebIO.Asset[WebIO.Asset("js", "knockout", "/home/dushya/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), WebIO.Asset("js", "knockout_punches", "/home/dushya/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), WebIO.Asset("js", nothing, "/home/dushya/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), WebIO.Asset("css", nothing, "/home/dushya/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), WebIO.Asset("css", nothing, "/home/dushya/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{WebIO.AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x00007f05e5af4970, Task (runnable) @0x00007f05e5af4970), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139665157062752, 0)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            )